<font face="微软雅黑" color=green size=5>SparkDataFrame基础</font>

In [1]:
from pyspark.ml.linalg import Vectors
from pyspark.sql import SparkSession
import pyspark.pandas as ps
import pyspark.sql.functions as F
from pyspark.sql.functions import monotonically_increasing_id
import findspark
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
spark_home = "D:\\Anaconda\\Lib\\site-packages\\pyspark"
python_path = "D:\\Anaconda\\python"
findspark.init(spark_home,python_path)
spark = SparkSession.builder.appName('SparkDataFrame').getOrCreate()

D:\Anaconda\lib\site-packages\pyspark\pandas\__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


# 读取SparkDataFrame

In [2]:
df_iris = spark.read.csv("../data/iris.csv", header=True, inferSchema=True)
#header参数设置为True，表示第一行包含列名。inferSchema参数设置为True，表示Spark将尝试推断每列的数据类型。

In [3]:
df_iris.printSchema() #查看数据的结构和列的数据类型

root
 |-- sepallength: double (nullable = true)
 |-- sepalwidth: double (nullable = true)
 |-- petallength: double (nullable = true)
 |-- petalwidth: double (nullable = true)
 |-- label: integer (nullable = true)



In [4]:
df_iris = df_iris.withColumn("label", df_iris["label"].cast("float"))#转换数据类型
df_iris.printSchema()

root
 |-- sepallength: double (nullable = true)
 |-- sepalwidth: double (nullable = true)
 |-- petallength: double (nullable = true)
 |-- petalwidth: double (nullable = true)
 |-- label: float (nullable = true)



# 创建SparkDataFrame

## 使用RDD来创建

主要使用RDD的toDF方法。

In [7]:
data = [("Sam", 28, 88), ("Flora", 28, 90), ("Run", 1, 60)] #list中的每一个元素都是元组
rdd = spark.sparkContext.parallelize(data)
dfFromRDD1 = rdd.toDF(["name", "age", "score"])
dfFromRDD1.show()
dfFromRDD1.printSchema()

+-----+---+-----+
| name|age|score|
+-----+---+-----+
|  Sam| 28|   88|
|Flora| 28|   90|
|  Run|  1|   60|
+-----+---+-----+

root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- score: long (nullable = true)



## 使用python的DataFrame来创建

In [19]:
#第一种
import pandas as pd

df = pd.DataFrame([['Sam', 28, 88], ['Flora', 28, 90], ['Run', 1, 60]],
                  columns=['name', 'age', 'score'])
print(df)
Spark_df = spark.createDataFrame(df)
Spark_df.show()

    name  age  score
0    Sam   28     88
1  Flora   28     90
2    Run    1     60
+-----+---+-----+
| name|age|score|
+-----+---+-----+
|  Sam| 28|   88|
|Flora| 28|   90|
|  Run|  1|   60|
+-----+---+-----+



In [21]:
#第二种
import pyspark.pandas as ps
Spark_df = ps.from_pandas(df).to_spark()
Spark_df.show()

+-----+---+-----+
| name|age|score|
+-----+---+-----+
|  Sam| 28|   88|
|Flora| 28|   90|
|  Run|  1|   60|
+-----+---+-----+



## 使用pyspark.pandas来创建

In [7]:
import pyspark.pandas as ps
import findspark
import warnings
warnings.filterwarnings('ignore')
list_values = [['Sam', 28, 88], ['Flora', 28, 90], ['Run', 1, 60]]
Spark_df = ps.DataFrame(data=list_values, columns=['name', 'age', 'score']).to_spark()

### 创建空DataFrame

In [11]:
a = ps.DataFrame(data=[], columns=['name', 'age', 'score']).to_spark()

In [12]:
a.collect()

[]

## 使用List来创建

In [30]:
list_values = [['Sam', 28, 88], ['Flora', 28, 90], ['Run', 1, None]]
Spark_df = spark.createDataFrame(list_values, ['name', 'age', 'score'])
Spark_df.show()

+-----+---+-----+
| name|age|score|
+-----+---+-----+
|  Sam| 28|   88|
|Flora| 28|   90|
|  Run|  1| null|
+-----+---+-----+



## 读取数据文件来创建

In [28]:
df = spark.read.option("header", "true")\
    .option("inferSchema", "true")\
    .option("delimiter", ",")\
    .csv("../data/iris.csv")
df.show(5)
df.printSchema()

+-----------+----------+-----------+----------+-----+
|sepallength|sepalwidth|petallength|petalwidth|label|
+-----------+----------+-----------+----------+-----+
|        5.1|       3.5|        1.4|      null|    0|
|       null|       3.0|        1.4|       0.2|    0|
|        4.7|       3.2|       null|       0.2|    0|
|        4.6|       3.1|       null|       0.2|    0|
|        5.0|       3.6|        1.4|       0.2|    0|
+-----------+----------+-----------+----------+-----+
only showing top 5 rows

root
 |-- sepallength: double (nullable = true)
 |-- sepalwidth: double (nullable = true)
 |-- petallength: double (nullable = true)
 |-- petalwidth: double (nullable = true)
 |-- label: integer (nullable = true)



spark.read.csv(path=data, encoding='gbk', quote='"', header='True', multiLine=True)
- path:文件名
- encoding:编码方式
- quote:忽略数据中的双引号，如果不忽略会错误解析
- header:是否有表头
- multiLine:允许字段跨越多行

## 通过读取数据库来创建

In [13]:
# url = "jdbc:mysql://192.168.11.138:3306/ai_monitor"
url = "jdbc:mysql://localhost:3306/sys"

df = spark.read.format("jdbc") \
 .option("url", url) \
 .option("dbtable", "data_gt1_cms_201508_201604") \
 .option("user", "root") \
 .option("password", "root") \
 .load()

In [15]:
df.show()

+--------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+
|            ts|      DWATT|       CSGV|       CTIM|    TTXD1_1|    TTXD1_2|    TTXD1_3|    TTXD1_4|    TTXD1_5|    TTXD1_6|    TTXD1_7|    TTXD1_8|    TTXD1_9|   TTXD1_10|   TTXD1_11|   TTXD1_12|   TTXD1_13|   TTXD1_14|   TTXD1_15|   TTXD1_16|   TTXD1_17|   TTXD1_18|   TTXD1_19|   TTXD1_20|   TTXD1_21|   TTXD1_22|   TTXD1_23|   TTXD1_24|       TTXM|     TTXSP1|     TTXSP3|     TTXSP2|
+--------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+--

# 保存SparkDataFrame

在将DataFrame保存为CSV文件时，需要注意以下事项：

- 输出文件夹或文件路径必须是一个不存在的路径，否则将会抛出错误。
- 如果输出文件夹已经存在，Spark会自动为保存的CSV文件添加后缀以避免文件名冲突。
- 默认情况下，每个CSV文件的文件名将为part-00000、part-00001等，可以在write.csv方法中通过option("pathGlobFilter", "*.csv")设置文件名。
- 如果DataFrame包含NULL值，则在保存为CSV文件时，NULL值将被保存为空字符串。
- 如果DataFrame的列中包含特殊字符或引号，记得在保存CSV文件时使用适当的转义字符。

In [4]:
df_iris = spark.read.csv("../data/iris.csv", header=True, inferSchema=True)
df_iris.show(5)

+-----------+----------+-----------+----------+-----+
|sepallength|sepalwidth|petallength|petalwidth|label|
+-----------+----------+-----------+----------+-----+
|        5.1|       3.5|        1.4|       0.2|    0|
|        4.9|       3.0|        1.4|       0.2|    0|
|        4.7|       3.2|        1.3|       0.2|    0|
|        4.6|       3.1|        1.5|       0.2|    0|
|        5.0|       3.6|        1.4|       0.2|    0|
+-----------+----------+-----------+----------+-----+
only showing top 5 rows



## 保存DataFrame为多个CSV文件

In [5]:
save_path = "../data/iris"
df_iris.write.csv(save_path, header=True)

In [ ]:
df_iris.read.csv(save_path, header=True)

## 保存DataFrame为单个CSV文件

In [6]:
save_path = "../data/iris_single"
df_iris.coalesce(1).write.csv(save_path, header=True)

In [ ]:
df_iris.read.csv(save_path, header=True)

# 常用的SparkDataFrame API

## 数据集

In [6]:
# 创建一个SparkDataFrame
rdd = spark.sparkContext.parallelize([("Sam", 28, 88., "M"),
                                      ("Flora", 28, 90., "F"),
                                      ("Run", 1, 60., None),
                                      ("Peter", 55, 100., "M"),
                                      ("Mei", 54, 95., "F"),
                                      ("Lei", 59, 65., "F"),
                                      ("Sam1", 28, 88., "M"),
                                      ("Flora1", 28, 90., "F"),
                                      ("Run1", 1, 60., None),
                                      ("Peter1", 55., 100, "M"),
                                      ("Mei1", 54, 95., "F"),
                                      ("Lei1", 59, 65., "F")])
df = rdd.toDF(["name", "age", "score", "sex"])
df.show()
df.printSchema()

+------+----+-----+----+
|  name| age|score| sex|
+------+----+-----+----+
|   Sam|  28| 88.0|   M|
| Flora|  28| 90.0|   F|
|   Run|   1| 60.0|null|
| Peter|  55|100.0|   M|
|   Mei|  54| 95.0|   F|
|   Lei|  59| 65.0|   F|
|  Sam1|  28| 88.0|   M|
|Flora1|  28| 90.0|   F|
|  Run1|   1| 60.0|null|
|Peter1|null| null|   M|
|  Mei1|  54| 95.0|   F|
|  Lei1|  59| 65.0|   F|
+------+----+-----+----+

root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- score: double (nullable = true)
 |-- sex: string (nullable = true)



## DataFrame的APIs

### 查看类型

In [15]:
df.dtypes

[('name', 'string'), ('age', 'bigint'), ('score', 'double'), ('sex', 'string')]

In [22]:
df.select('name').schema[0]

StructField('name', StringType(), True)

### 修改类型

In [11]:
df.select('name').dtypes[0][1] == 'string'

True

In [16]:
df = df.withColumn("age", F.col("age").cast("double"))
df.show()

+------+----+-----+----+
|  name| age|score| sex|
+------+----+-----+----+
|   Sam|28.0| 88.0|   M|
| Flora|28.0| 90.0|   F|
|   Run| 1.0| 60.0|null|
| Peter|55.0|100.0|   M|
|   Mei|54.0| 95.0|   F|
|   Lei|59.0| 65.0|   F|
|  Sam1|28.0| 88.0|   M|
|Flora1|28.0| 90.0|   F|
|  Run1| 1.0| 60.0|null|
|Peter1|null| null|   M|
|  Mei1|54.0| 95.0|   F|
|  Lei1|59.0| 65.0|   F|
+------+----+-----+----+



In [ ]:
# 修改为时间列
df.withColumn(time_col, F.to_timestamp(F.col(time_col), 'yyyy-MM-dd HH:mm:ss'))

### 查询行

In [6]:
#指定要打印的行数
df.show()
df.show(3)

+-----+---+-----+----+
| name|age|score| sex|
+-----+---+-----+----+
|  Sam| 28|   88|   M|
|Flora| 28|   90|   F|
|  Run|  1|   60|null|
|Peter| 55|  100|   M|
|  Mei| 54|   95|   F|
+-----+---+-----+----+

+-----+---+-----+----+
| name|age|score| sex|
+-----+---+-----+----+
|  Sam| 28|   88|   M|
|Flora| 28|   90|   F|
|  Run|  1|   60|null|
+-----+---+-----+----+
only showing top 3 rows



In [9]:
# 以列表形式返回行
df.collect()

[Row(name='Sam', age=28, score=88, sex='M'),
 Row(name='Flora', age=28, score=90, sex='F'),
 Row(name='Run', age=1, score=60, sex=None),
 Row(name='Peter', age=55, score=100, sex='M'),
 Row(name='Mei', age=54, score=95, sex='F')]

In [21]:
#查询总行数
df.count()

5

In [7]:
# 查看第1条数据
df.first()

Row(name='Sam', age=28, score=88, sex='M')

In [8]:
#获取头几行到本地
df.head(2)
df.take(2)

[Row(name='Sam', age=28, score=88, sex='M'),
 Row(name='Flora', age=28, score=90, sex='F')]

In [22]:
# 获取后几行
df.tail(3)

[Row(name='Haffman', department='Marketing', salary=7000),
 Row(name='Ilaja', department='Marketing', salary=8000),
 Row(name='Joey', department='Sales', salary=9000)]

In [9]:
# 按照一定规则从df随机抽样数据
df.sample(0.5).show()

+-----+---+-----+---+
| name|age|score|sex|
+-----+---+-----+---+
|Flora| 28|   90|  F|
|Peter| 55|  100|  M|
|  Mei| 54|   95|  F|
+-----+---+-----+---+



### 查询列

In [3]:
#查询列名
df.columns

['name', 'age', 'score', 'sex']

In [5]:
list(set(df.columns) - set(['name']))

['sex', 'age', 'score']

In [50]:
#查询列类型
df.dtypes

[('name', 'string'), ('age', 'bigint'), ('score', 'bigint'), ('sex', 'string')]

In [44]:
# 返回列的基础统计信息
df.describe(['age']).show()

+-------+------------------+
|summary|               age|
+-------+------------------+
|  count|                 5|
|   mean|              33.2|
| stddev|22.353970564532826|
|    min|                 1|
|    max|                55|
+-------+------------------+



In [43]:
#查询概况
df.describe().show()

+-------+-----+------------------+------------------+----+
|summary| name|               age|             score| sex|
+-------+-----+------------------+------------------+----+
|  count|    5|                 5|                 5|   4|
|   mean| null|              33.2|              86.6|null|
| stddev| null|22.353970564532826|15.582040944625966|null|
|    min|Flora|                 1|                60|   F|
|    max|  Sam|                55|               100|   M|
+-------+-----+------------------+------------------+----+



In [45]:
# 选定指定列并按照一定顺序呈现
df.select("sex", "score").show()

+----+-----+
| sex|score|
+----+-----+
|   M|   88|
|   F|   90|
|null|   60|
|   M|  100|
|   F|   95|
+----+-----+



In [51]:
# 查看指定列的枚举值
df.freqItems(["age","sex"]).show()

+---------------+-------------+
|  age_freqItems|sex_freqItems|
+---------------+-------------+
|[55, 1, 28, 54]| [M, null, F]|
+---------------+-------------+



In [52]:
df.summary().show()

+-------+-----+------------------+------------------+----+
|summary| name|               age|             score| sex|
+-------+-----+------------------+------------------+----+
|  count|    5|                 5|                 5|   4|
|   mean| null|              33.2|              86.6|null|
| stddev| null|22.353970564532826|15.582040944625966|null|
|    min|Flora|                 1|                60|   F|
|    25%| null|                28|                88|null|
|    50%| null|                28|                90|null|
|    75%| null|                54|                95|null|
|    max|  Sam|                55|               100|   M|
+-------+-----+------------------+------------------+----+



### 选择行

In [59]:
#选择dataframe中间的特定行数，该函数是不确定的，因为它的结果取决于分区 ID。
from pyspark.sql.functions import monotonically_increasing_id #设定自增长列
dfWithIndex = df.withColumn('id',monotonically_increasing_id()) #添加自增长列
dfWithIndex.select(dfWithIndex.name, dfWithIndex.id.between(2, 4)).show()

+-----+-------------------------+
| name|((id >= 2) AND (id <= 4))|
+-----+-------------------------+
|  Sam|                    false|
|Flora|                    false|
|  Run|                    false|
|Peter|                    false|
|  Mei|                    false|
+-----+-------------------------+



In [62]:
# 使用`zipWithIndex`方法和`toDF`函数添加递增列
df_with_incremental_values = df.rdd.zipWithIndex().map(lambda x: x[0] + (x[1],)).toDF(df.columns + ['id'])

# 显示结果
df_with_incremental_values.show()
df_with_incremental_values.select(df_with_incremental_values.name, df_with_incremental_values.id.between(2, 4)).show()

+-----+---+-----+----+---+
| name|age|score| sex| id|
+-----+---+-----+----+---+
|  Sam| 28|   88|   M|  0|
|Flora| 28|   90|   F|  1|
|  Run|  1|   60|null|  2|
|Peter| 55|  100|   M|  3|
|  Mei| 54|   95|   F|  4|
+-----+---+-----+----+---+

+-----+-------------------------+
| name|((id >= 2) AND (id <= 4))|
+-----+-------------------------+
|  Sam|                    false|
|Flora|                    false|
|  Run|                     true|
|Peter|                     true|
|  Mei|                     true|
+-----+-------------------------+



### 选择列

In [15]:
dfs = df.select('name','age')
dfs.collect()

[Row(name='Sam', age=28),
 Row(name='Flora', age=28),
 Row(name='Run', age=1),
 Row(name='Peter', age=55),
 Row(name='Mei', age=54)]

In [24]:
age = df.select('age').collect()
list_age = [i[0] for i in age]

In [25]:
list_age

[28, 28, 1, 55, 54]

In [22]:
dfs.where(F.col('name') == 'Mei').collect()[0]['age']

54

In [27]:
df.select('age').collect()[4]['age']

54

In [7]:
# 选择符合类型的列
df1 = df.pandas_api()
df1.select_dtypes(include=['float']).columns # 包含

Index(['score'], dtype='object')

In [8]:
# 选择符合类型的列
df2 = df.pandas_api()
df2.select_dtypes(exclude=['string', 'datetime', 'datetime64', 'bool']).columns # 不包含

Index(['age', 'score'], dtype='object')

### 排序

In [54]:
#降序
df.orderBy(df['age'].desc()).show(5)

+-----+---+-----+----+
| name|age|score| sex|
+-----+---+-----+----+
|Peter| 55|  100|   M|
|  Mei| 54|   95|   F|
|  Sam| 28|   88|   M|
|Flora| 28|   90|   F|
|  Run|  1|   60|null|
+-----+---+-----+----+



In [19]:
# 降序只取一条
df.orderBy(df['age'].desc()).limit(1).show()

+-----+---+-----+---+
| name|age|score|sex|
+-----+---+-----+---+
|Peter| 55|  100|  M|
+-----+---+-----+---+



In [55]:
#升序
df.orderBy(df['age']).show(5)

+-----+---+-----+----+
| name|age|score| sex|
+-----+---+-----+----+
|  Run|  1|   60|null|
|  Sam| 28|   88|   M|
|Flora| 28|   90|   F|
|  Mei| 54|   95|   F|
|Peter| 55|  100|   M|
+-----+---+-----+----+



In [3]:
# 升序
df.orderBy(['age'], ascending=True).show()

+------+----+-----+----+
|  name| age|score| sex|
+------+----+-----+----+
|Peter1|null| null|   M|
|   Run|   1| 60.0|null|
|  Run1|   1| 60.0|null|
| Flora|  28| 90.0|   F|
|   Sam|  28| 88.0|   M|
|  Sam1|  28| 88.0|   M|
|Flora1|  28| 90.0|   F|
|  Mei1|  54| 95.0|   F|
|   Mei|  54| 95.0|   F|
| Peter|  55|100.0|   M|
|   Lei|  59| 65.0|   F|
|  Lei1|  59| 65.0|   F|
+------+----+-----+----+



In [4]:
# 分组排序，先对age升序，再对socre分组降序
sort_columns = ['age', 'score']
ascending_columns = [True, False]
df.orderBy(sort_columns, ascending=ascending_columns).show()

+------+----+-----+----+
|  name| age|score| sex|
+------+----+-----+----+
|Peter1|null| null|   M|
|  Run1|   1| 60.0|null|
|   Run|   1| 60.0|null|
| Flora|  28| 90.0|   F|
|Flora1|  28| 90.0|   F|
|  Sam1|  28| 88.0|   M|
|   Sam|  28| 88.0|   M|
|  Mei1|  54| 95.0|   F|
|   Mei|  54| 95.0|   F|
| Peter|  55|100.0|   M|
|   Lei|  59| 65.0|   F|
|  Lei1|  59| 65.0|   F|
+------+----+-----+----+



### 去重

In [63]:
# 对数据集进行去重
df.distinct().show()

+-----+---+-----+----+
| name|age|score| sex|
+-----+---+-----+----+
|  Sam| 28|   88|   M|
|Flora| 28|   90|   F|
|  Run|  1|   60|null|
|Peter| 55|  100|   M|
|  Mei| 54|   95|   F|
+-----+---+-----+----+



In [65]:
# 对指定列去重
df.dropDuplicates(["sex"]).show()

+-----+---+-----+----+
| name|age|score| sex|
+-----+---+-----+----+
|  Run|  1|   60|null|
|Flora| 28|   90|   F|
|  Sam| 28|   88|   M|
+-----+---+-----+----+



In [66]:
# 根据指定的df对df进行去重
df1 = spark.createDataFrame(
        [("a", 1), ("a", 1), ("b",  3), ("c", 4)], ["C1", "C2"])
df2 = spark.createDataFrame([("a", 1), ("b", 3)], ["C1", "C2"])
df3 = df1.exceptAll(df2)  # 没有去重的功效
df4 = df1.subtract(df2)  # 有去重的奇效
df1.show()
df2.show()
df3.show()
df4.show()

+---+---+
| C1| C2|
+---+---+
|  a|  1|
|  a|  1|
|  b|  3|
|  c|  4|
+---+---+

+---+---+
| C1| C2|
+---+---+
|  a|  1|
|  b|  3|
+---+---+

+---+---+
| C1| C2|
+---+---+
|  a|  1|
|  c|  4|
+---+---+

+---+---+
| C1| C2|
+---+---+
|  c|  4|
+---+---+



In [67]:
# 返回两个DataFrame的交集
df1 = spark.createDataFrame(
        [("a", 1), ("a", 1), ("b",  3), ("c", 4)], ["C1", "C2"])
df2 = spark.createDataFrame([("a", 1), ("b", 4)], ["C1", "C2"])
df1.intersectAll(df2).show()

+---+---+
| C1| C2|
+---+---+
|  a|  1|
+---+---+



### 丢弃列

In [68]:
# 丢弃指定列
df.drop('age').show()

+-----+-----+----+
| name|score| sex|
+-----+-----+----+
|  Sam|   88|   M|
|Flora|   90|   F|
|  Run|   60|null|
|Peter|  100|   M|
|  Mei|   95|   F|
+-----+-----+----+



In [69]:
# 丢弃空值
df.dropna(how='all', subset=['sex']).show()

+-----+---+-----+---+
| name|age|score|sex|
+-----+---+-----+---+
|  Sam| 28|   88|  M|
|Flora| 28|   90|  F|
|Peter| 55|  100|  M|
|  Mei| 54|   95|  F|
+-----+---+-----+---+



### 新增列

#### append

每次添加一个值。

In [33]:
expressions = [] # 创建一个空列表用以存储表达式
res = F.col('age') - F.col('score') # 计算表达式
expressions.append(res.alias('res1')) # 修改列名并添加到数组中
res = res + 100 # 表达式运算
expressions.append(res.alias('res2')) # 修改列名并添加到数组中
df_resi = df.select(*expressions) # 获取数组结果

In [30]:
df_resi.show()

+-----+----+
| res1|res2|
+-----+----+
|-60.0|40.0|
|-62.0|38.0|
|-59.0|41.0|
|-45.0|55.0|
|-41.0|59.0|
| -6.0|94.0|
|-60.0|40.0|
|-62.0|38.0|
|-59.0|41.0|
| null|null|
|-41.0|59.0|
| -6.0|94.0|
+-----+----+



In [34]:
expressions.extend(df.columns) # 原始列
df_resi = df.select(*expressions) # 获取数组结果
df_resi.show()

+-----+----+------+----+-----+----+
| res1|res2|  name| age|score| sex|
+-----+----+------+----+-----+----+
|-60.0|40.0|   Sam|  28| 88.0|   M|
|-62.0|38.0| Flora|  28| 90.0|   F|
|-59.0|41.0|   Run|   1| 60.0|null|
|-45.0|55.0| Peter|  55|100.0|   M|
|-41.0|59.0|   Mei|  54| 95.0|   F|
| -6.0|94.0|   Lei|  59| 65.0|   F|
|-60.0|40.0|  Sam1|  28| 88.0|   M|
|-62.0|38.0|Flora1|  28| 90.0|   F|
|-59.0|41.0|  Run1|   1| 60.0|null|
| null|null|Peter1|null| null|   M|
|-41.0|59.0|  Mei1|  54| 95.0|   F|
| -6.0|94.0|  Lei1|  59| 65.0|   F|
+-----+----+------+----+-----+----+



#### extend

每次可添加一个数组。

In [19]:
expressions = [] # 创建一个空列表用以存储表达式
res1 = F.col('age') - F.col('score') # 计算表达式1
res2 = F.col('score') - F.col('age') # 计算表达式2
expressions.extend(df.columns) # 原始列
expressions.extend([res1.alias('res1'), res2.alias('res2')]) # 修改列名并添加到数组中
df_resi = df.select(*expressions) # 获取数组结果

In [20]:
df_resi.show()

+------+----+-----+----+-----+----+
|  name| age|score| sex| res1|res2|
+------+----+-----+----+-----+----+
|   Sam|  28| 88.0|   M|-60.0|60.0|
| Flora|  28| 90.0|   F|-62.0|62.0|
|   Run|   1| 60.0|null|-59.0|59.0|
| Peter|  55|100.0|   M|-45.0|45.0|
|   Mei|  54| 95.0|   F|-41.0|41.0|
|   Lei|  59| 65.0|   F| -6.0| 6.0|
|  Sam1|  28| 88.0|   M|-60.0|60.0|
|Flora1|  28| 90.0|   F|-62.0|62.0|
|  Run1|   1| 60.0|null|-59.0|59.0|
|Peter1|null| null|   M| null|null|
|  Mei1|  54| 95.0|   F|-41.0|41.0|
|  Lei1|  59| 65.0|   F| -6.0| 6.0|
+------+----+-----+----+-----+----+



#### lit

In [5]:
from pyspark.sql.functions import lit
# 添加常量列，值都是相同的
df.withColumn("country", lit("China")).show()

+-----+---+-----+----+-------+
| name|age|score| sex|country|
+-----+---+-----+----+-------+
|  Sam| 28|   88|   M|  China|
|Flora| 28|   90|   F|  China|
|  Run|  1|   60|null|  China|
|Peter| 55|  100|   M|  China|
|  Mei| 54|   95|   F|  China|
+-----+---+-----+----+-------+



In [15]:
# 添加统计列
df.withColumn("sum", lit(df.agg(F.sum('score')).collect()[0][0])).show()

+-----+---+-----+----+---+
| name|age|score| sex|sum|
+-----+---+-----+----+---+
|  Sam| 28|   88|   M|433|
|Flora| 28|   90|   F|433|
|  Run|  1|   60|null|433|
|Peter| 55|  100|   M|433|
|  Mei| 54|   95|   F|433|
+-----+---+-----+----+---+



In [10]:
# 增加年龄列的值
df.withColumn("new_age", F.col("age") + lit(5)).show()

+-----+---+-----+----+-------+
| name|age|score| sex|new_age|
+-----+---+-----+----+-------+
|  Sam| 28|   88|   M|     33|
|Flora| 28|   90|   F|     33|
|  Run|  1|   60|null|      6|
|Peter| 55|  100|   M|     60|
|  Mei| 54|   95|   F|     59|
+-----+---+-----+----+-------+



##### 列值替换

In [7]:
# 列替换值
df.withColumn("country", lit("USA")).show()

+-----+---+-----+----+-------+
| name|age|score| sex|country|
+-----+---+-----+----+-------+
|  Sam| 28|   88|   M|    USA|
|Flora| 28|   90|   F|    USA|
|  Run|  1|   60|null|    USA|
|Peter| 55|  100|   M|    USA|
|  Mei| 54|   95|   F|    USA|
+-----+---+-----+----+-------+



#### withColumn

**尽量不要用withColumn，会导致内存变大，运行速度变慢，可以用append代替。**

In [70]:
# 新增相关列，新增的列和之前列有关
df1 = df.withColumn("birth_year", 2021 - df.age)
df1.show()

+-----+---+-----+----+----------+
| name|age|score| sex|birth_year|
+-----+---+-----+----+----------+
|  Sam| 28|   88|   M|      1993|
|Flora| 28|   90|   F|      1993|
|  Run|  1|   60|null|      2020|
|Peter| 55|  100|   M|      1966|
|  Mei| 54|   95|   F|      1967|
+-----+---+-----+----+----------+



#### monotonically_increasing_id

In [3]:
# 创建另一个SparkDataFrame
list_values = [["China"],["USA"],["China"],["UK"],["India"]]
df1 = spark.createDataFrame(list_values, ["country"])
df1.show()

+-------+
|country|
+-------+
|  China|
|    USA|
|  China|
|     UK|
|  India|
+-------+



In [27]:
#新增无关列，新增的列和之前列无关
#先设置自增长列，根据自增长列的序号join，最后匹配，自增长列和分块有关，不同块值不同，并不能完全自增长
temp = df.withColumn('Row_ID', monotonically_increasing_id())
temp1 = df1.withColumn('Row_ID', monotonically_increasing_id())
temp2 = temp.join(temp1, temp['Row_ID'] == temp1['Row_ID'], 'left').drop('Row_ID')
temp2.show()

+-----+---+-----+----+-------+
| name|age|score| sex|country|
+-----+---+-----+----+-------+
|  Sam| 28|   88|   M|  China|
|Flora| 28|   90|   F|    USA|
|  Run|  1|   60|null|  China|
|Peter| 55|  100|   M|     UK|
|  Mei| 54|   95|   F|  India|
+-----+---+-----+----+-------+



#### zipWithIndex

In [6]:
# 使用`zipWithIndex`方法和`toDF`函数添加递增列
temp = df.rdd.zipWithIndex().map(lambda x: x[0] + (x[1],)).toDF(df.columns + ['Row_ID'])
temp1 = df1.rdd.zipWithIndex().map(lambda x: x[0] + (x[1],)).toDF(df1.columns + ['Row_ID'])
temp2 = temp.join(temp1, on=['Row_ID'], how='left').drop('Row_ID')
temp2.show()

+-----+---+-----+----+-------+
| name|age|score| sex|country|
+-----+---+-----+----+-------+
|  Sam| 28|   88|   M|  China|
|Flora| 28|   90|   F|    USA|
|  Run|  1|   60|null|  China|
|Peter| 55|  100|   M|     UK|
|  Mei| 54|   95|   F|  India|
+-----+---+-----+----+-------+



### 合并列

In [12]:
df1 = df.withColumn("age_score", F.concat_ws('_', F.col("age"), F.col("score")))
df1.show()

+-----+---+-----+----+---------+
| name|age|score| sex|age_score|
+-----+---+-----+----+---------+
|  Sam| 28|   88|   M|    28_88|
|Flora| 28|   90|   F|    28_90|
|  Run|  1|   60|null|     1_60|
|Peter| 55|  100|   M|   55_100|
|  Mei| 54|   95|   F|    54_95|
+-----+---+-----+----+---------+



In [5]:
df2 = df.withColumn("age_score",F.concat("age","score"))
df2.show()

+-----+---+-----+----+---------+
| name|age|score| sex|age_score|
+-----+---+-----+----+---------+
|  Sam| 28|   88|   M|     2888|
|Flora| 28|   90|   F|     2890|
|  Run|  1|   60|null|      160|
|Peter| 55|  100|   M|    55100|
|  Mei| 54|   95|   F|     5495|
+-----+---+-----+----+---------+



### 重命名列

In [71]:
# 重命名列名
df1 = df.withColumnRenamed("sex", "gender")
df1.show()

+-----+---+-----+------+
| name|age|score|gender|
+-----+---+-----+------+
|  Sam| 28|   88|     M|
|Flora| 28|   90|     F|
|  Run|  1|   60|  null|
|Peter| 55|  100|     M|
|  Mei| 54|   95|     F|
+-----+---+-----+------+



In [ ]:
#取别名
df['age'].alias('age_value')
df.select(df['age'].alias('age_value'),'name').show()

### 修改列类型

In [23]:
df.withColumn("age", df["age"].cast('string'))
df.show()

+-----+---+-----+----+
| name|age|score| sex|
+-----+---+-----+----+
|  Sam| 28|   88|   M|
|Flora| 28|   90|   F|
|  Run|  1|   60|null|
|Peter| 55|  100|   M|
|  Mei| 54|   95|   F|
+-----+---+-----+----+



In [33]:
df.withColumn("age", df["age"].astype('string'))
df.show()

+-----+---+-----+----+
| name|age|score| sex|
+-----+---+-----+----+
|  Sam| 28|   88|   M|
|Flora| 28|   90|   F|
|  Run|  1|   60|null|
|Peter| 55|  100|   M|
|  Mei| 54|   95|   F|
+-----+---+-----+----+



### 填充列

In [72]:
# 空值填充操作
df1 = spark.createDataFrame(
        [("a", None), ("a", 1), (None,  3), ("c", 4)], ["C1", "C2"])
# df2 = df1.na.fill({"C1": "d", "C2": 99})
df2 = df1.fillna({"C1": "d", "C2": 99})
df1.show()
df2.show()

+----+----+
|  C1|  C2|
+----+----+
|   a|null|
|   a|   1|
|null|   3|
|   c|   4|
+----+----+

+---+---+
| C1| C2|
+---+---+
|  a| 99|
|  a|  1|
|  d|  3|
|  c|  4|
+---+---+



### 条件过滤

#### first

In [11]:
df.select(F.first(F.col("age"))).collect()[0][0]

28

In [14]:
df.first()

Row(name='Sam', age=28, score=88.0, sex='M')

#### last

In [12]:
df.select(F.last(F.col("age"))).collect()[0][0]

59

#### filter

In [35]:
# 根据条件过滤
df.filter(df.age.between(50, 90)).show()

+-----+---+-----+---+
| name|age|score|sex|
+-----+---+-----+---+
|Peter| 55|  100|  M|
|  Mei| 54|   95|  F|
+-----+---+-----+---+



In [45]:
# 可以使用正则的匹配
df.filter(df.name.rlike('Me$')).show()

+----+---+-----+---+
|name|age|score|sex|
+----+---+-----+---+
+----+---+-----+---+



In [43]:
# 返回含有关键词的行
df.filter(df.name.like('Mei')).show()

+----+---+-----+---+
|name|age|score|sex|
+----+---+-----+---+
| Mei| 54|   95|  F|
+----+---+-----+---+



In [38]:
# 筛选非空的行
df.filter(df.sex.isNull()).show()

+----+---+-----+----+
|name|age|score| sex|
+----+---+-----+----+
| Run|  1|   60|null|
+----+---+-----+----+



In [37]:
# 筛选非空的行
df.filter(df.sex.isNotNull()).show()

+-----+---+-----+---+
| name|age|score|sex|
+-----+---+-----+---+
|  Sam| 28|   88|  M|
|Flora| 28|   90|  F|
|Peter| 55|  100|  M|
|  Mei| 54|   95|  F|
+-----+---+-----+---+



In [36]:
# 是否包含某个关键词
df.filter(df.age.contains(55)).show()

+-----+---+-----+---+
| name|age|score|sex|
+-----+---+-----+---+
|Peter| 55|  100|  M|
+-----+---+-----+---+



In [34]:
#单条件过滤
df.filter(df.age>50).show()

+-----+---+-----+---+
| name|age|score|sex|
+-----+---+-----+---+
|Peter| 55|  100|  M|
|  Mei| 54|   95|  F|
+-----+---+-----+---+



In [ ]:
#多条件过滤
df.filter((df.age>50) | (df.age<10)).show() 

In [75]:
df.filter("age<18").show()

+----+---+-----+----+
|name|age|score| sex|
+----+---+-----+----+
| Run|  1|   60|null|
+----+---+-----+----+



#### take

In [54]:
# 读取前两行
df.take(2)

[Row(name='Sam', age=28, score=88, sex='M'),
 Row(name='Flora', age=28, score=90, sex='F')]

#### where

In [3]:
#条件选择
df.where(df.age==28).show()

+-----+---+-----+---+
| name|age|score|sex|
+-----+---+-----+---+
|  Sam| 28|   88|  M|
|Flora| 28|   90|  F|
+-----+---+-----+---+



In [17]:
# 多条件选择，并且只能用&，|符号
df.where((df.age==28)|(df.score==88)).show()

+-----+---+-----+---+
| name|age|score|sex|
+-----+---+-----+---+
|  Sam| 28|   88|  M|
|Flora| 28|   90|  F|
+-----+---+-----+---+



#### when

In [21]:
#when(condition, value1).otherwise(value2)联合使用：
#当满足条件condition的指赋值为values1,不满足条件的则赋值为values2.
#otherwise表示，不满足条件的情况下，应该赋值为啥
df.select(df.name, df.age, F.when(df.age > 4, 1).when(df.age < 3, -1).otherwise(0).alias("判断")).show()

+-----+---+----+
| name|age|判断|
+-----+---+----+
|  Sam| 28|   1|
|Flora| 28|   1|
|  Run|  1|  -1|
|Peter| 55|   1|
|  Mei| 54|   1|
+-----+---+----+



In [16]:
#多条件需要用括号括起来，并且只能用&，|符号
df.select(df.name, df.age, F.when((df.age > 4)&(df.age < 2), 1).when(df.age < 3, -1).otherwise(0).alias("判断")).show()

+-----+---+----+
| name|age|判断|
+-----+---+----+
|  Sam| 28|   0|
|Flora| 28|   0|
|  Run|  1|  -1|
|Peter| 55|   0|
|  Mei| 54|   0|
+-----+---+----+



In [ ]:
df.withColumn("value_th", F.when(df["age"] >= 10, 1).when(df["value"] <= 20, -1).otherwise(0))

#### isin

In [17]:
# 列表参数
filter_list = ["Sam", "Flora"]

# 使用isin函数进行筛选
filtered_df = df.filter(F.col("name").isin(filter_list))
filtered_df.show()

+-----+---+-----+---+
| name|age|score|sex|
+-----+---+-----+---+
|  Sam| 28|   88|  M|
|Flora| 28|   90|  F|
+-----+---+-----+---+



In [40]:
# 返回包含某些值的行
df.filter(df.name.isin('Peter', 'Mei')).show()

+-----+---+-----+---+
| name|age|score|sex|
+-----+---+-----+---+
|Peter| 55|  100|  M|
|  Mei| 54|   95|  F|
+-----+---+-----+---+



#### isnull

In [32]:
# pyspark中DataFrame的缺失值是None类型的，可用F.isnull()来判定。
df.select(F.isnull(df['name'])).show()

+--------------+
|(name IS NULL)|
+--------------+
|         false|
|         false|
|         false|
|         false|
|         false|
+--------------+



In [33]:
df1 = df.filter(F.isnull("sex"))
df1.show()

+----+---+-----+----+
|name|age|score| sex|
+----+---+-----+----+
| Run|  1|   60|null|
+----+---+-----+----+



#### coalesce

In [20]:
#返回第一个不为null的列
df.withColumn('age_name',F.coalesce(df.age,df.name)).show()

+-----+---+-----+----+--------+
| name|age|score| sex|age_name|
+-----+---+-----+----+--------+
|  Sam| 28|   88|   M|      28|
|Flora| 28|   90|   F|      28|
|  Run|  1|   60|null|       1|
|Peter| 55|  100|   M|      55|
|  Mei| 54|   95|   F|      54|
+-----+---+-----+----+--------+



#### expr

In [22]:
#expr()方法解析给定的SQL表达式
df.select(F.expr('upper(name)')).show() #字母大写

+-----------+
|upper(name)|
+-----------+
|        SAM|
|      FLORA|
|        RUN|
|      PETER|
|        MEI|
+-----------+



In [23]:
#expr(~) 方法通常可以使用 PySpark DataFrame 的 selectExpr(~) 方法编写得更简洁
df.selectExpr('upper(name)').show()

+-----------+
|upper(name)|
+-----------+
|        SAM|
|      FLORA|
|        RUN|
|      PETER|
|        MEI|
+-----------+



In [24]:
#使用expr方法解析复杂的SQL表达式
df.select(F.expr('age > 20 AND name LIKE "S%"').alias('result')).show()

+------+
|result|
+------+
|  true|
| false|
| false|
| false|
| false|
+------+



In [26]:
col = F.expr('CASE WHEN age < 40 THEN "1" ELSE "0" END').alias('result')
df.withColumn('status', col).show()

+-----+---+-----+----+------+
| name|age|score| sex|status|
+-----+---+-----+----+------+
|  Sam| 28|   88|   M|     1|
|Flora| 28|   90|   F|     1|
|  Run|  1|   60|null|     1|
|Peter| 55|  100|   M|     0|
|  Mei| 54|   95|   F|     0|
+-----+---+-----+----+------+



#### substring

In [28]:
df.withColumn("age_1", F.substring(df.name, 2, 3)).show()

+-----+---+-----+----+-----+
| name|age|score| sex|age_1|
+-----+---+-----+----+-----+
|  Sam| 28|   88|   M|   am|
|Flora| 28|   90|   F|  lor|
|  Run|  1|   60|null|   un|
|Peter| 55|  100|   M|  ete|
|  Mei| 54|   95|   F|   ei|
+-----+---+-----+----+-----+



### join

In [15]:
df1 = spark.createDataFrame([("a", 1), ("d", 1), ("b", 3), ("c", 4)],
                            ["id", "num1"])
df2 = spark.createDataFrame([("a", 1), ("b", 3)], ["id", "num2"])
df1.join(df2, df1.id == df2.id, 'left').select(df1.id.alias("df1_id"),
                                               df1.num1.alias("df1_num"),
                                               df2.num2.alias("df2_num")
                                               ).sort(["df1_id"], ascending=False)\
    .show()

+------+-------+-------+
|df1_id|df1_num|df2_num|
+------+-------+-------+
|     d|      1|   null|
|     c|      4|   null|
|     b|      3|      3|
|     a|      1|      1|
+------+-------+-------+



### 聚合函数

- avg(\*cols)     ——   计算每组中一列或多列的平均值
- count()          ——   计算每组中一共有多少行，返回DataFrame有2列，一列为分组的组名，另一列为行总数
- max(\*cols)    ——   计算每组中一列或多列的最大值
- mean(\*cols)  ——  计算每组中一列或多列的平均值
- min(\*cols)     ——  计算每组中一列或多列的最小值
- sum(\*cols)    ——   计算每组中一列或多列的总和

In [7]:
# 根据某一列进行聚合
df.agg(
    F.min(df.age).alias("最小年龄"),
    F.expr("avg(age)").alias("平均年龄"),
).show()

+--------+--------+
|最小年龄|平均年龄|
+--------+--------+
|       1|    33.2|
+--------+--------+



### 分组聚合

In [5]:
# 根据某几列进行聚合，如有多列用列表写在一起，如 df.groupBy(["sex", "age"])
df.groupBy("sex").agg(F.min(df.age).alias("最小年龄"),
                      F.expr("avg(age)").alias("平均年龄"),
                      F.expr("collect_list(name)").alias("姓名集合")
                      ).show()

+----+--------+--------+------------+
| sex|最小年龄|平均年龄|    姓名集合|
+----+--------+--------+------------+
|   M|      28|    41.5|[Sam, Peter]|
|   F|      28|    41.0|[Flora, Mei]|
|null|       1|     1.0|       [Run]|
+----+--------+--------+------------+



In [12]:
# 对每一行进行函数方法的应用
def f(person):
    print(person.name)
df.foreach(f)

In [13]:
# 修改df里的某些值
df1 = df.na.replace({"M": "Male", "F": "Female"})
df1.show()

+-----+---+-----+------+
| name|age|score|   sex|
+-----+---+-----+------+
|  Sam| 28|   88|  Male|
|Flora| 28|   90|Female|
|  Run|  1|   60|  null|
|Peter| 55|  100|  Male|
|  Mei| 54|   95|Female|
+-----+---+-----+------+



In [8]:
#计算中位数
df.approxQuantile("age", [0.5], 0.01)

[28.0]

### union

In [3]:
# 相当于SQL里的union all操作
df1 = spark.createDataFrame(
        [("a", 1), ("d", 1), ("b",  3), ("c", 4)], ["id", "num"])
df2 = spark.createDataFrame([("a", 1), ("b", 3)], ["id", "num"])
df1.union(df2).show()
df1.unionAll(df2).show()

+---+---+
| id|num|
+---+---+
|  a|  1|
|  d|  1|
|  b|  3|
|  c|  4|
|  a|  1|
|  b|  3|
+---+---+

+---+---+
| id|num|
+---+---+
|  a|  1|
|  d|  1|
|  b|  3|
|  c|  4|
|  a|  1|
|  b|  3|
+---+---+



In [5]:
# 根据列名来进行合并数据集
df1 = spark.createDataFrame([[1, 2, 3]], ["col0", "col1", "col2"])
df2 = spark.createDataFrame([[4, 5, 6]], ["col1", "col2", "col0"])
df1.unionByName(df2).show()

+----+----+----+
|col0|col1|col2|
+----+----+----+
|   1|   2|   3|
|   6|   4|   5|
+----+----+----+



### 集合

In [18]:
sentenceDataFrame = spark.createDataFrame(
    ((1, "asf"), (2, "2143"), (3, "rfds"))).toDF("label", "sentence")
sentenceDataFrame.show()

sentenceDataFrame1 = spark.createDataFrame(
    ((1, "asf"), (2, "2143"), (4, "f8934y"))).toDF("label", "sentence")
sentenceDataFrame1.show()

+-----+--------+
|label|sentence|
+-----+--------+
|    1|     asf|
|    2|    2143|
|    3|    rfds|
+-----+--------+

+-----+--------+
|label|sentence|
+-----+--------+
|    1|     asf|
|    2|    2143|
|    4|  f8934y|
+-----+--------+



In [17]:
# 差集
newDF = sentenceDataFrame1.select("sentence").subtract(sentenceDataFrame.select("sentence"))
newDF.show()

+--------+
|sentence|
+--------+
|  f8934y|
+--------+



In [19]:
# 交集
newDF = sentenceDataFrame1.select("sentence").intersect(sentenceDataFrame.select("sentence"))
newDF.show()

+--------+
|sentence|
+--------+
|     asf|
|    2143|
+--------+



In [20]:
# 并集
newDF = sentenceDataFrame1.select("sentence").union(sentenceDataFrame.select("sentence"))
newDF.show()

+--------+
|sentence|
+--------+
|     asf|
|    2143|
|  f8934y|
|     asf|
|    2143|
|    rfds|
+--------+



In [21]:
# 并集 + 去重
newDF = sentenceDataFrame1.select("sentence").union(sentenceDataFrame.select("sentence")).distinct()
newDF.show()

+--------+
|sentence|
+--------+
|     asf|
|    2143|
|  f8934y|
|    rfds|
+--------+



### 列名替换

In [ ]:
data = data.pandas_api()
data.columns = data.columns.str.replace(".", "#")
data = data.to_spark()

## 自定义函数

- 在spark 1.x 版本的时候，利用pyspark包中的 udf() 来开发用户自定义函数，自定义的函数只能接收单一数值，因此当你向自定义函数中传递 dataframe的一个列时，自定义函数内部的处理方式就是执行了for循环，将传入列中的每个值逐一处理，这样效率比较低，在加上scala进程与python进程间数据传递、序列化与反序列化的时间消耗，用户自定义函数的执行效率比较低。**单点类型**
- 在spark 2.x 版本中，以上的问题有了很大改善，新增了用户自定义函数的定义接口pandas_udf，并且对scala进程与python进程间的数据传递过程做了优化，数据从scala进程到python进程时，需要进行序列化，而当python进程执行完毕，将数据传递回scala进行时，则无需进行反序列化，这是因为使用了Apache Arrow 这种基于内存的列式数据存储格式，这样就节省了时间，当然需要将spark参数spark.sql.execution.arrow.enabled设置为true。总的来说，目前在实际开发中，用到的基本都是pandas_udf了。**pandas.Series类型**

**用户自定义的python函数是会提交到python进程中执行的，因此函数中使用的应该是python和各个包(pyspark之外)的函数与数对象，而不是pyspark中的函数与数据对象。**

### udf

#### 匿名函数

In [3]:
func = F.udf(lambda x: x+1)
df1 = df.withColumn('age+1',func(df["age"]))
df1.show()

+-----+---+-----+----+-----+
| name|age|score| sex|age+1|
+-----+---+-----+----+-----+
|  Sam| 28|   88|   M|   29|
|Flora| 28|   90|   F|   29|
|  Run|  1|   60|null|    2|
|Peter| 55|  100|   M|   56|
|  Mei| 54|   95|   F|   55|
+-----+---+-----+----+-----+



In [9]:
from pyspark.sql.functions import udf
#Use udf to define a row-at-a-time udf
@udf()
# Input/output are both a single double value
def plus_one(x):
    return x + 1
df1 = df.withColumn('age+1',plus_one(df.age))
df1.show()

+-----+---+-----+----+-----+
| name|age|score| sex|age+1|
+-----+---+-----+----+-----+
|  Sam| 28|   88|   M|   29|
|Flora| 28|   90|   F|   29|
|  Run|  1|   60|null|    2|
|Peter| 55|  100|   M|   56|
|  Mei| 54|   95|   F|   55|
+-----+---+-----+----+-----+



#### 普通函数

In [6]:
# 定义一个函数来计算平方
def score(x):
    if x > 80 and x < 90:
        return 'B'
    elif x >= 90:
        return 'A'
    else:
        return 'C'

# 注册UDF
score_udf = F.udf(score)

# 使用UDF对DataFrame进行操作
df1 = df.withColumn('degree', score_udf(df['score']))
df1.show()

+-----+---+-----+----+------+
| name|age|score| sex|degree|
+-----+---+-----+----+------+
|  Sam| 28|   88|   M|     B|
|Flora| 28|   90|   F|     A|
|  Run|  1|   60|null|     C|
|Peter| 55|  100|   M|     A|
|  Mei| 54|   95|   F|     A|
+-----+---+-----+----+------+



In [10]:
@udf()
# 定义一个函数来计算平方
def score(x):
    if x > 80 and x < 90:
        return 'B'
    elif x >= 90:
        return 'A'
    else:
        return 'C'

# 使用UDF对DataFrame进行操作
df1 = df.withColumn('degree', score(df['score']))
df1.show()

+-----+---+-----+----+------+
| name|age|score| sex|degree|
+-----+---+-----+----+------+
|  Sam| 28|   88|   M|     B|
|Flora| 28|   90|   F|     A|
|  Run|  1|   60|null|     C|
|Peter| 55|  100|   M|     A|
|  Mei| 54|   95|   F|     A|
+-----+---+-----+----+------+



### pandas_udf

In [4]:
from pyspark.sql.functions import pandas_udf, PandasUDFType

pandas_udf函数支持定义三种类型的函数，分别为SCALAR、GROUP_MAP、GROUP_AGG，在pyspark.sql.function.PandasUDFType 中做了定义，默认SCALAR类型。

#### SCALAR 类型

SCALAR 类型自定义函数可以**接收多个列的输入，也只能接受列输入，但永远只会输出一个列**，并且返回的数据类型是pyspark.sql.types 中定义的数据类型。应用SCALAR类型自定义函数时，在自定义函数内部应当执行的一些逻辑不那么复杂的操作，如果不是这样，你可以使用其它的方式来满足需求。例如当你需要对每一条数据用不同的、比较复杂的逻辑来处理，那可以用 GROUP_MAP 类型来自定义函数，或者将dataframe转换为rdd，然后用mapValue函数来处理。

该类型支持用户自定义向量型操作的函数，也就是pandas、numpy中的向量操作，传入的格式为pd.Series，传出的格式为pd.Series。

使用scipy去计算一个值的累计正太分布概率值，stats.norm.cdf 可以使用在一个标量或者是一个pandas.Series上面。

In [24]:
import pandas as pd
from scipy import stats

@pandas_udf(returnType='double')
def cdf(v):
    return pd.Series(stats.norm.cdf(v))

df1 = df.withColumn('cumulative_probability',cdf(df.age))
df1.show()

+-----+---+-----+----+----------------------+
| name|age|score| sex|cumulative_probability|
+-----+---+-----+----+----------------------+
|  Sam| 28|   88|   M|                   1.0|
|Flora| 28|   90|   F|                   1.0|
|  Run|  1|   60|null|    0.8413447460685429|
|Peter| 55|  100|   M|                   1.0|
|  Mei| 54|   95|   F|                   1.0|
+-----+---+-----+----+----------------------+



#### GROUP_MAP 类型

该类型支持用户将dataframe按照一组字段分组，然后对分组后的多个dataframe分别用户自定义函数处理，在用户自定义函数中，处理的对象是pandas 的DataFrame。

**该方法在@pandas_udf的输入列和输出列必须一致，类型也必须一致，否则将报错。**

##### 多参数udf

In [5]:
# pandas版本的函数，直接传入DataFrame
def func(df, column):
    df[column] = df[column] + 3
    df.sort_values(by=[column], inplace=True, ignore_index=True)
    return df

# 增加一列不变列，用于在分组的时候将数据划分到一起
df = df.withColumn('id', F.lit(1))

# 依次使'age', 'score'列分别加3
for i in ['age', 'score']:
    # 装饰函数，注册UDF
    @pandas_udf(df.schema, PandasUDFType.GROUPED_MAP)
    def func_udf(df):
        return func(df, i)
    df = df.groupby('id').apply(func_udf)
df = df.drop('id')
df.show()

D:\Anaconda\lib\site-packages\pyspark\sql\pandas\group_ops.py:104: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


+------+----+-----+----+
|  name| age|score| sex|
+------+----+-----+----+
|   Run|   4| 63.0|NULL|
|  Run1|   4| 63.0|NULL|
|   Lei|  62| 68.0|   F|
|  Lei1|  62| 68.0|   F|
|   Sam|  31| 91.0|   M|
|  Sam1|  31| 91.0|   M|
| Flora|  31| 93.0|   F|
|Flora1|  31| 93.0|   F|
|   Mei|  57| 98.0|   F|
|  Mei1|  57| 98.0|   F|
| Peter|  58|103.0|   M|
|Peter1|NULL| NULL|   M|
+------+----+-----+----+



##### 单参数udf

In [15]:
# pandas版本的函数，直接传入DataFrame
def func(df):
    return df

# 增加一列不变列，用于在分组的时候将数据划分到一起
df = df.withColumn('id', F.lit(1))

# 装饰函数，注册UDF
@pandas_udf(df.schema, PandasUDFType.GROUPED_MAP)
def func_udf(df):
    return func(df)
df = df.groupby('id').apply(func_udf)
df = df.drop('id')
df.show()

D:\Anaconda\lib\site-packages\pyspark\sql\pandas\group_ops.py:104: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


+----+------+
| age|income|
+----+------+
| 1.0|  NULL|
| 2.0|  NULL|
| 2.0|  NULL|
|NULL|   3.0|
| 4.0|   4.0|
| 5.0|   5.0|
+----+------+



#### GROUP_AGG 类型

这种类型用得不多，因为可以直接被`dataframe.groupby().agg(sf.min())` 这样的函数取代。它的功能就是分组计算统计值。

In [10]:
from pyspark.sql.types import StringType,IntegerType,StructType,StructField
def func(x):
    return x.mean()

func_udf = pandas_udf(func, IntegerType(), PandasUDFType.GROUPED_AGG)
df1 = df.groupby('age').agg(func_udf(df['score']))
df1.show()

D:\Anaconda\Lib\site-packages\pyspark\sql\pandas\functions.py:399: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


+---+-----------+
|age|func(score)|
+---+-----------+
|  1|         60|
| 28|         89|
| 54|         95|
| 55|        100|
+---+-----------+



## 格式转换

### 转pandas

In [25]:
df.toPandas()

,name,age,score,sex
0,Sam,28,88,M
1,Flora,28,90,F
2,Run,1,60,None
3,Peter,55,100,M
4,Mei,54,95,F


转化为pandas，但是该数据要读入内存，如果数据量大的话，很难跑得动

两者的异同：

- Pyspark DataFrame是在分布式节点上运行一些数据操作，而pandas是不可能的；
- Pyspark DataFrame的数据反映比较缓慢，没有Pandas那么及时反映；
- Pyspark DataFrame的数据框是不可变的，不能任意添加列，只能通过合并进行；
- pandas比Pyspark DataFrame有更多方便的操作以及很强大

如果包含时间标签，需要先将时间格式转为字符串  
`df = df.withColumn("ts", df["ts"].cast('string'))`

In [10]:
df.pandas_api()

,name,age,score,sex
0,Sam,28.0,88.0,M
1,Flora,28.0,90.0,F
2,Run,1.0,60.0,None
3,Peter,55.0,100.0,M
4,Mei,54.0,95.0,F
5,Lei,59.0,65.0,F
6,Sam1,28.0,88.0,M
7,Flora1,28.0,90.0,F
8,Run1,1.0,60.0,None
9,Peter1,NaN,NaN,M


### 转spark

In [30]:
import pyspark.pandas as ps
df1 = df.toPandas()
df = ps.DataFrame(df1).to_spark()
df.show()

D:\Anaconda\Lib\site-packages\pyspark\pandas\utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


+-----+---+-----+----+
| name|age|score| sex|
+-----+---+-----+----+
|  Sam| 28|   88|   M|
|Flora| 28|   90|   F|
|  Run|  1|   60|null|
|Peter| 55|  100|   M|
|  Mei| 54|   95|   F|
+-----+---+-----+----+



### 转rdd

In [32]:
rdd_df = df.rdd
df = rdd_df.toDF()
df.show()

+-----+---+-----+----+
| name|age|score| sex|
+-----+---+-----+----+
|  Sam| 28|   88|   M|
|Flora| 28|   90|   F|
|  Run|  1|   60|null|
|Peter| 55|  100|   M|
|  Mei| 54|   95|   F|
+-----+---+-----+----+



## Pandas_UDF与toPandas的区别

@pandas_udf 创建一个向量化的用户定义函数(UDF)，利用了panda的矢量化特性，是udf的一种更快的替代方案，因此适用于分布式数据集。

toPandas将分布式spark数据集转换为pandas数据集，对pandas数据集进行本地化，并且所有数据都驻留在驱动程序内存中，因此此方法仅在预期生成的pandas DataFrame较小的情况下使用。

换句话说，@pandas_udf使用panda API来处理分布式数据集，而toPandas()将分布式数据集转换为本地数据，然后使用pandas进行处理。

# SparkDataFrame时间处理

**SparkDataFrame支持两种时间格式，包括：'date'和'timestamp'。**

In [12]:
data=[["1","2020-02-03"],["2","2019-03-05"],["3","2021-03-09"]]
df=spark.createDataFrame(data, ["id","time"])
df.show()

+---+----------+
| id|      time|
+---+----------+
|  1|2020-02-03|
|  2|2019-03-05|
|  3|2021-03-09|
+---+----------+



## 日期

### 当前日期

获取当前系统日期。默认情况下，数据将以yyyy-dd-mm格式返回。

In [6]:
df.select(F.current_date().alias("current_date")).show(1)

+------------+
|current_date|
+------------+
|  2023-11-09|
+------------+
only showing top 1 row



### 日期格式

解析日期并转换yyyy-dd-mm为MM-dd-yyyy格式。

In [7]:
df.select(F.col("time"), F.date_format(F.col("time"), "MM-dd-yyyy").alias("date_format")).show()

+----------+-----------+
|      time|date_format|
+----------+-----------+
|2020-02-01| 02-01-2020|
|2019-03-01| 03-01-2019|
|2021-03-01| 03-01-2021|
+----------+-----------+



### 字符串转日期格式

In [8]:
df.select(F.col("time"), F.to_date(F.col("time"), "yyyy-MM-dd HH:mm:ss").alias("to_date")).show()

+----------+----------+
|      time|   to_date|
+----------+----------+
|2020-02-01|2020-02-01|
|2019-03-01|2019-03-01|
|2021-03-01|2021-03-01|
+----------+----------+



### 日期之差

可以直接两个时间戳相减。

### 天数之差

In [9]:
df.select(F.col("time"), F.datediff(F.current_date(), F.col("time")).alias("datediff")).show()

+----------+--------+
|      time|datediff|
+----------+--------+
|2020-02-01|    1377|
|2019-03-01|    1714|
|2021-03-01|     983|
+----------+--------+



### 月数只差

In [10]:
df.select(F.col("time"), F.months_between(F.current_date(),F.col("time")).alias("months_between")  ).show()

+----------+--------------+
|      time|months_between|
+----------+--------------+
|2020-02-01|   45.25806452|
|2019-03-01|   56.25806452|
|2021-03-01|   32.25806452|
+----------+--------------+



### 截断指定单位的日期

In [13]:
df.select(F.col("time"),
          F.trunc(F.col("time"), "Month").alias("Month_Trunc"),
          F.trunc(F.col("time"), "Year").alias("Month_Year"),
          F.trunc(F.col("time"), "Month").alias("Month_Trunc")).show()

+----------+-----------+----------+-----------+
|      time|Month_Trunc|Month_Year|Month_Trunc|
+----------+-----------+----------+-----------+
|2020-02-03| 2020-02-01|2020-01-01| 2020-02-01|
|2019-03-05| 2019-03-01|2019-01-01| 2019-03-01|
|2021-03-09| 2021-03-01|2021-01-01| 2021-03-01|
+----------+-----------+----------+-----------+



### 月、日加减法

In [14]:
df.select(F.col("time"),
          F.add_months(F.col("time"), 3).alias("add_months"),
          F.add_months(F.col("time"), -3).alias("sub_months"),
          F.date_add(F.col("time"), 4).alias("date_add"),
          F.date_sub(F.col("time"), 4).alias("date_sub")).show()

+----------+----------+----------+----------+----------+
|      time|add_months|sub_months|  date_add|  date_sub|
+----------+----------+----------+----------+----------+
|2020-02-03|2020-05-03|2019-11-03|2020-02-07|2020-01-30|
|2019-03-05|2019-06-05|2018-12-05|2019-03-09|2019-03-01|
|2021-03-09|2021-06-09|2020-12-09|2021-03-13|2021-03-05|
+----------+----------+----------+----------+----------+



### 年、月、下一天、一年中第几个星期

In [15]:
df.select(F.col("time"),
          F.year(F.col("time")).alias("year"),
          F.month(F.col("time")).alias("month"),
          F.next_day(F.col("time"), "Sunday").alias("next_day"),
          F.weekofyear(F.col("time")).alias("weekofyear")).show()

+----------+----+-----+----------+----------+
|      time|year|month|  next_day|weekofyear|
+----------+----+-----+----------+----------+
|2020-02-03|2020|    2|2020-02-09|         6|
|2019-03-05|2019|    3|2019-03-10|        10|
|2021-03-09|2021|    3|2021-03-14|        10|
+----------+----+-----+----------+----------+



### 星期几、月日、年日

- 查询星期几
- 一个月中的第几天
- 一年中的第几天

In [16]:
df.select(
    F.col("time"),
    F.dayofweek(F.col("time")).alias("dayofweek"),
    F.dayofmonth(F.col("time")).alias("dayofmonth"),
    F.dayofyear(F.col("time")).alias("dayofyear"),
).show()

+----------+---------+----------+---------+
|      time|dayofweek|dayofmonth|dayofyear|
+----------+---------+----------+---------+
|2020-02-03|        2|         3|       34|
|2019-03-05|        3|         5|       64|
|2021-03-09|        3|         9|       68|
+----------+---------+----------+---------+



## 时间

In [17]:
data = [["1", "02-01-2020 11 01 19 06"], ["2", "03-01-2019 12 01 19 406"],
        ["3", "03-01-2021 12 01 19 406"]]
df2 = spark.createDataFrame(data, ["id", "time"])
df2.show(truncate=False)

+---+-----------------------+
|id |time                   |
+---+-----------------------+
|1  |02-01-2020 11 01 19 06 |
|2  |03-01-2019 12 01 19 406|
|3  |03-01-2021 12 01 19 406|
+---+-----------------------+



### 返回当前时间戳

In [18]:
#默认格式yyyy-MM-dd HH:mm:ss
df2.select(F.current_timestamp().alias("current_timestamp")).show()

+--------------------+
|   current_timestamp|
+--------------------+
|2023-11-09 18:00:...|
|2023-11-09 18:00:...|
|2023-11-09 18:00:...|
+--------------------+



### 字符串转为时间戳

In [19]:
df2.select(
    F.col("time"),
    F.to_timestamp(F.col("time"),"MM-dd-yyyy HH mm ss SSS").alias("to_timestamp")).show(truncate=False)

+-----------------------+-----------------------+
|time                   |to_timestamp           |
+-----------------------+-----------------------+
|02-01-2020 11 01 19 06 |2020-02-01 11:01:19.06 |
|03-01-2019 12 01 19 406|2019-03-01 12:01:19.406|
|03-01-2021 12 01 19 406|2021-03-01 12:01:19.406|
+-----------------------+-----------------------+



### 获取小时、分钟、秒

In [20]:
# 数据
data = [["1", "2020-02-01 11:01:19.06"], ["2", "2019-03-01 12:01:19.406"],
        ["3", "2021-03-01 12:01:19.406"]]
df3 = spark.createDataFrame(data, ["id", "time"])

# 提取小时、分钟、秒
df3.select(F.col("time"),
           F.hour(F.col("time")).alias("hour"),
           F.minute(F.col("time")).alias("minute"),
           F.second(F.col("time")).alias("second")).show(truncate=False)

+-----------------------+----+------+------+
|time                   |hour|minute|second|
+-----------------------+----+------+------+
|2020-02-01 11:01:19.06 |11  |1     |19    |
|2019-03-01 12:01:19.406|12  |1     |19    |
|2021-03-01 12:01:19.406|12  |1     |19    |
+-----------------------+----+------+------+



## 滑动窗口

In [21]:
from pyspark.sql.window import Window
import pyspark.sql.functions as F

Window函数分类为三种:
- 排名函数 ranking functions包括:
  - row_number()
  - rank()
  - dense_rank()
  - percent_rank()
  - ntile()
  - orderBy()
- 解析函数 analytic functions包括:
  - cume_dist()
  - lag()
  - lead()
- 聚合函数 aggregate functions包括:
  - sum()
  - first()
  - last()
  - max()
  - min()
  - mean()
  - stddev()

### 创建一个 PySpark DataFrame

In [20]:
employee_salary = [
    ("Ali", "Sales", 8000),
    ("Bob", "Sales", 7000),
    ("Cindy", "Sales", 7500),
    ("Davd", "Finance", 10000),
    ("Elena", "Sales", 8000),
    ("Fancy", "Finance", 12000),
    ("George", "Finance", 11000),
    ("Haffman", "Marketing", 7000),
    ("Ilaja", "Marketing", 8000),
    ("Joey", "Sales", 9000)]
 
columns= ["name", "department", "salary"]
df = spark.createDataFrame(data = employee_salary, schema = columns)
df.show(truncate=False)

+-------+----------+------+
|name   |department|salary|
+-------+----------+------+
|Ali    |Sales     |8000  |
|Bob    |Sales     |7000  |
|Cindy  |Sales     |7500  |
|Davd   |Finance   |10000 |
|Elena  |Sales     |8000  |
|Fancy  |Finance   |12000 |
|George |Finance   |11000 |
|Haffman|Marketing |7000  |
|Ilaja  |Marketing |8000  |
|Joey   |Sales     |9000  |
+-------+----------+------+



### 窗口函数 ranking functions

#### row_number()

row_number() 窗口函数用于给出从1开始到每个窗口分区的结果的连续行号。 与 groupBy 不同 Window 以 partitionBy 作为分组条件，orderBy 对 Window 分组内的数据进行排序。

In [10]:
# 以 department 字段进行分组，以 salary 倒序排序
# 按照部门对薪水排名，薪水最低的为第一名
windowSpec = Window.partitionBy("department").orderBy(F.asc("salary"))
# 分组内增加 row_number
df_part = df.withColumn(
    "row_number", 
    F.row_number().over(windowSpec)
)
df_part.show()

+-------+----------+------+----------+
|   name|department|salary|row_number|
+-------+----------+------+----------+
|   Davd|   Finance| 10000|         1|
| George|   Finance| 11000|         2|
|  Fancy|   Finance| 12000|         3|
|Haffman| Marketing|  7000|         1|
|  Ilaja| Marketing|  8000|         2|
|    Bob|     Sales|  7000|         1|
|  Cindy|     Sales|  7500|         2|
|    Ali|     Sales|  8000|         3|
|  Elena|     Sales|  8000|         4|
|   Joey|     Sales|  9000|         5|
+-------+----------+------+----------+



In [11]:
df_part.where(F.col('row_number') == 2).show()

+------+----------+------+----------+
|  name|department|salary|row_number|
+------+----------+------+----------+
|George|   Finance| 11000|         2|
| Ilaja| Marketing|  8000|         2|
| Cindy|     Sales|  7500|         2|
+------+----------+------+----------+



#### rank()

rank()用来给按照指定列排序的分组窗增加一个排序的序号，

如果有相同数值，则排序数相同，下一个序数顺延一位。来看如下代码：

In [12]:
# 使用 rank 排序，都是8000的薪水，就同列第二
windowSpec = Window.partitionBy("department").orderBy(F.desc("salary"))
df_rank = df.withColumn("rank", F.rank().over(windowSpec))
df_rank.show()

+-------+----------+------+----+
|   name|department|salary|rank|
+-------+----------+------+----+
|  Fancy|   Finance| 12000|   1|
| George|   Finance| 11000|   2|
|   Davd|   Finance| 10000|   3|
|  Ilaja| Marketing|  8000|   1|
|Haffman| Marketing|  7000|   2|
|   Joey|     Sales|  9000|   1|
|    Ali|     Sales|  8000|   2|
|  Elena|     Sales|  8000|   2|
|  Cindy|     Sales|  7500|   4|
|    Bob|     Sales|  7000|   5|
+-------+----------+------+----+



In [13]:
df_rank.where(F.col("rank")=="2").show()

+-------+----------+------+----+
|   name|department|salary|rank|
+-------+----------+------+----+
| George|   Finance| 11000|   2|
|Haffman| Marketing|  7000|   2|
|    Ali|     Sales|  8000|   2|
|  Elena|     Sales|  8000|   2|
+-------+----------+------+----+



#### dense_rank()

In [14]:
# 注意 rank 排序，8000虽然为同列第二，但7500属于第4名
# dense_rank()中， 8000同列第二后，7500属于第3名
windowSpec  = Window.partitionBy("department").orderBy(F.desc("salary"))
df.withColumn("dense_rank", F.dense_rank().over(windowSpec)).show()

+-------+----------+------+----------+
|   name|department|salary|dense_rank|
+-------+----------+------+----------+
|  Fancy|   Finance| 12000|         1|
| George|   Finance| 11000|         2|
|   Davd|   Finance| 10000|         3|
|  Ilaja| Marketing|  8000|         1|
|Haffman| Marketing|  7000|         2|
|   Joey|     Sales|  9000|         1|
|    Ali|     Sales|  8000|         2|
|  Elena|     Sales|  8000|         2|
|  Cindy|     Sales|  7500|         3|
|    Bob|     Sales|  7000|         4|
+-------+----------+------+----------+



#### percent_rank()

计算不同数值的百分比排序数据

In [15]:
windowSpec  = Window.partitionBy("department").orderBy(F.desc("salary"))
df.withColumn("percent_rank",F.percent_rank().over(windowSpec)).show()

+-------+----------+------+------------+
|   name|department|salary|percent_rank|
+-------+----------+------+------------+
|  Fancy|   Finance| 12000|         0.0|
| George|   Finance| 11000|         0.5|
|   Davd|   Finance| 10000|         1.0|
|  Ilaja| Marketing|  8000|         0.0|
|Haffman| Marketing|  7000|         1.0|
|   Joey|     Sales|  9000|         0.0|
|    Ali|     Sales|  8000|        0.25|
|  Elena|     Sales|  8000|        0.25|
|  Cindy|     Sales|  7500|        0.75|
|    Bob|     Sales|  7000|         1.0|
+-------+----------+------+------------+



上述结果可以理解为将 dense_rank() 的结果进行归一化， 即可得到0-1以内的百分数。percent_rank() 与 SQL 中的 PERCENT_RANK 函数效果一致。

#### ntile()

ntile()可将分组的数据按照指定数值n切分为n个部分， 每一部分按照行的先后给定相同的序数。例如n指定为2，则将组内数据分为两个部分， 第一部分序号为1，第二部分序号为2。理论上两部分数据行数是均等的， 但当数据为奇数行时，中间的那一行归到前一部分。

按照部门对数据进行分组，然后在组内按照薪水高低进行排序， 再使用 ntile() 将组内数据切分为两个部分。结果如下：

In [16]:
# 按照部门对数据进行分组，然后在组内按照薪水高低进行排序 
windowSpec = Window.partitionBy(
    "department").orderBy(F.desc("salary"))
# 使用ntile() 将组内数据切分为两个部分
df.withColumn("ntile", F.ntile(2).over(windowSpec)).show()

+-------+----------+------+-----+
|   name|department|salary|ntile|
+-------+----------+------+-----+
|  Fancy|   Finance| 12000|    1|
| George|   Finance| 11000|    1|
|   Davd|   Finance| 10000|    2|
|  Ilaja| Marketing|  8000|    1|
|Haffman| Marketing|  7000|    2|
|   Joey|     Sales|  9000|    1|
|    Ali|     Sales|  8000|    1|
|  Elena|     Sales|  8000|    1|
|  Cindy|     Sales|  7500|    2|
|    Bob|     Sales|  7000|    2|
+-------+----------+------+-----+



###  分析函数 Analytic functions

#### cume_dist()

cume_dist()函数用来获取数值的累进分布值，看如下例子：

In [17]:
windowSpec = Window.partitionBy("department").orderBy(F.desc("salary"))
df.withColumn("cume_dist", F.cume_dist().over(windowSpec)).show()

+-------+----------+------+------------------+
|   name|department|salary|         cume_dist|
+-------+----------+------+------------------+
|  Fancy|   Finance| 12000|0.3333333333333333|
| George|   Finance| 11000|0.6666666666666666|
|   Davd|   Finance| 10000|               1.0|
|  Ilaja| Marketing|  8000|               0.5|
|Haffman| Marketing|  7000|               1.0|
|   Joey|     Sales|  9000|               0.2|
|    Ali|     Sales|  8000|               0.6|
|  Elena|     Sales|  8000|               0.6|
|  Cindy|     Sales|  7500|               0.8|
|    Bob|     Sales|  7000|               1.0|
+-------+----------+------+------------------+



In [18]:
# 和 percent_rank 对比一下
df.withColumn(
    'percent_rank',
    F.percent_rank().over(windowSpec)).show()

+-------+----------+------+------------+
|   name|department|salary|percent_rank|
+-------+----------+------+------------+
|  Fancy|   Finance| 12000|         0.0|
| George|   Finance| 11000|         0.5|
|   Davd|   Finance| 10000|         1.0|
|  Ilaja| Marketing|  8000|         0.0|
|Haffman| Marketing|  7000|         1.0|
|   Joey|     Sales|  9000|         0.0|
|    Ali|     Sales|  8000|        0.25|
|  Elena|     Sales|  8000|        0.25|
|  Cindy|     Sales|  7500|        0.75|
|    Bob|     Sales|  7000|         1.0|
+-------+----------+------+------------+



#### lag()

lag() 函数用于寻找按照指定列排好序的分组内每个数值的上一个数值，

通俗的说，就是数值排好序以后，寻找排在每个数值的上一个数值。代码如下：

In [19]:
# 相当于滞后项
windowSpec  = Window.partitionBy("department").orderBy(F.desc("salary"))
df.withColumn("lag", F.lag("salary",1).over(windowSpec)).show()

+-------+----------+------+-----+
|   name|department|salary|  lag|
+-------+----------+------+-----+
|  Fancy|   Finance| 12000| null|
| George|   Finance| 11000|12000|
|   Davd|   Finance| 10000|11000|
|  Ilaja| Marketing|  8000| null|
|Haffman| Marketing|  7000| 8000|
|   Joey|     Sales|  9000| null|
|    Ali|     Sales|  8000| 9000|
|  Elena|     Sales|  8000| 8000|
|  Cindy|     Sales|  7500| 8000|
|    Bob|     Sales|  7000| 7500|
+-------+----------+------+-----+



#### lead()

lead() 用于获取排序后的数值的下一个，代码如下：

In [20]:
# 和滞后项相反，提前一位
windowSpec  = Window.partitionBy("department").orderBy(F.desc("salary"))
df.withColumn("lead",F.lead("salary",1).over(windowSpec)).show()

+-------+----------+------+-----+
|   name|department|salary| lead|
+-------+----------+------+-----+
|  Fancy|   Finance| 12000|11000|
| George|   Finance| 11000|10000|
|   Davd|   Finance| 10000| null|
|  Ilaja| Marketing|  8000| 7000|
|Haffman| Marketing|  7000| null|
|   Joey|     Sales|  9000| 8000|
|    Ali|     Sales|  8000| 8000|
|  Elena|     Sales|  8000| 7500|
|  Cindy|     Sales|  7500| 7000|
|    Bob|     Sales|  7000| null|
+-------+----------+------+-----+



### 聚合函数 Aggregate Functions

常见的聚合函数有avg, sum, min, max, count, approx_count_distinct()等，我们用如下代码来同时使用这些函数：

In [21]:
# 分组，并对组内数据排序
windowSpec  = Window.partitionBy("department").orderBy(F.desc("salary"))
# 仅分组
windowSpecAgg  = Window.partitionBy("department")

df.withColumn("row", F.row_number().over(windowSpec)) \
  .withColumn("avg", F.avg("salary").over(windowSpecAgg)) \
  .withColumn("sum", F.sum("salary").over(windowSpecAgg)) \
  .withColumn("min", F.min("salary").over(windowSpecAgg)) \
  .withColumn("max", F.max("salary").over(windowSpecAgg)) \
  .withColumn("count", F.count("salary").over(windowSpecAgg)) \
  .withColumn("distinct_count", F.approxCountDistinct("salary").over(windowSpecAgg)) \
  .show()

D:\Anaconda\Lib\site-packages\pyspark\sql\functions.py:2610: FutureWarning: Deprecated in 2.1, use approx_count_distinct instead.
  warnings.warn("Deprecated in 2.1, use approx_count_distinct instead.", FutureWarning)


+-------+----------+------+---+-------+-----+-----+-----+-----+--------------+
|   name|department|salary|row|    avg|  sum|  min|  max|count|distinct_count|
+-------+----------+------+---+-------+-----+-----+-----+-----+--------------+
|  Fancy|   Finance| 12000|  1|11000.0|33000|10000|12000|    3|             3|
| George|   Finance| 11000|  2|11000.0|33000|10000|12000|    3|             3|
|   Davd|   Finance| 10000|  3|11000.0|33000|10000|12000|    3|             3|
|  Ilaja| Marketing|  8000|  1| 7500.0|15000| 7000| 8000|    2|             2|
|Haffman| Marketing|  7000|  2| 7500.0|15000| 7000| 8000|    2|             2|
|   Joey|     Sales|  9000|  1| 7900.0|39500| 7000| 9000|    5|             4|
|    Ali|     Sales|  8000|  2| 7900.0|39500| 7000| 9000|    5|             4|
|  Elena|     Sales|  8000|  3| 7900.0|39500| 7000| 9000|    5|             4|
|  Cindy|     Sales|  7500|  4| 7900.0|39500| 7000| 9000|    5|             4|
|    Bob|     Sales|  7000|  5| 7900.0|39500| 7000| 

需要注意的是 approx_count_distinct() 函数适用与窗函数的统计， 而在groupby中通常用countDistinct()来代替该函数，用来求组内不重复的数值的条数。

从结果来看，统计值基本上是按照部门分组，统计组内的salary情况。 如果我们只想要保留部门的统计结果，而将每个人的实际情况去掉，可以采用如下代码：

In [22]:
windowSpec  = Window.partitionBy("department").orderBy(F.desc("salary"))
windowSpecAgg  = Window.partitionBy("department")

df = df.withColumn("row", F.row_number().over(windowSpec)) \
  .withColumn("avg", F.avg("salary").over(windowSpecAgg)) \
  .withColumn("sum", F.sum("salary").over(windowSpecAgg)) \
  .withColumn("min", F.min("salary").over(windowSpecAgg)) \
  .withColumn("max", F.max("salary").over(windowSpecAgg)) \
  .withColumn("count", F.count("salary").over(windowSpecAgg)) \
  .withColumn("distinct_count", F.approx_count_distinct("salary").over(windowSpecAgg))

# 仅选取分组第一行数据
# 用F.col 去选row 行，怪怪的
df_part  = df.where(F.col("row")==1)
df_part.select("department","avg","sum","min","max","count","distinct_count").show()

+----------+-------+-----+-----+-----+-----+--------------+
|department|    avg|  sum|  min|  max|count|distinct_count|
+----------+-------+-----+-----+-----+-----+--------------+
|   Finance|11000.0|33000|10000|12000|    3|             3|
| Marketing| 7500.0|15000| 7000| 8000|    2|             2|
|     Sales| 7900.0|39500| 7000| 9000|    5|             4|
+----------+-------+-----+-----+-----+-----+--------------+



## 常用函数

In [6]:
# 加载和准备时间序列数据
data = [("2022-01-01", 10), ("2022-01-02", 15), ("2022-01-03", -25),
        ("2022-01-04", -25), ("2022-01-05", -30), ("2022-01-06", 10),
        ("2022-01-07", 40)]

df = spark.createDataFrame(data, ["timestamp", "value"])

In [8]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col
# 定义滑动窗口规范
windowSpec = Window.orderBy(F.col("timestamp")).rowsBetween(-4, 0)

# 应用窗口规范和转换操作
df1 = df.withColumn("moving_average", F.avg(F.col("value")).over(windowSpec))

# 显示结果
df1.show()

+----------+-----+--------------+
| timestamp|value|moving_average|
+----------+-----+--------------+
|2022-01-01|   10|          10.0|
|2022-01-02|   15|          12.5|
|2022-01-03|  -25|           0.0|
|2022-01-04|  -25|         -6.25|
|2022-01-05|  -30|         -11.0|
|2022-01-06|   10|         -11.0|
|2022-01-07|   40|          -6.0|
+----------+-----+--------------+



In [35]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col

# 加载和准备时间序列数据
data = [("2022-01-01", -10), ("2022-01-02", -15), ("2022-01-03", -25),
        ("2022-01-04", -25), ("2022-01-05", -30), ("2022-01-06", -10),
        ("2022-01-07", -40)]

df = spark.createDataFrame(data, ["timestamp", "value"])

# 定义滑动窗口规范
windowSpec = Window.orderBy(col("timestamp")).rowsBetween(-2, 2)

# 判定正负
windowSpec = Window.orderBy(col("timestamp")).rowsBetween(-2, 2)
sum_ = F.sum(F.col("value")).over(windowSpec)
sum_sum = F.abs(sum_)
sum_abs = F.sum(F.abs(F.col("value"))).over(windowSpec)
bu_column = F.when((sum_sum-sum_abs) == 0, F.abs(F.col("value"))).otherwise(F.col("value")).alias(f"value_bu")

# 显示结果
df.select(bu_column).show()

+--------+
|value_bu|
+--------+
|      10|
|      15|
|      25|
|      25|
|      30|
|      10|
|      40|
+--------+



# 缺失值处理

In [8]:
from pyspark.ml.feature import Imputer

In [9]:
df = spark.createDataFrame([(1.0, float("nan")), (2.0, float("nan")),
                            (2.0, float("nan")), (float("nan"), 3.0),
                            (4.0, 4.0), (5.0, 5.0)], ["age", "income"])
df.show()

+---+------+
|age|income|
+---+------+
|1.0|   NaN|
|2.0|   NaN|
|2.0|   NaN|
|NaN|   3.0|
|4.0|   4.0|
|5.0|   5.0|
+---+------+



## 删除缺失值的行

In [10]:
# 仅保留不包含缺失值的行
df1 = df.dropna()
df1.show()

+---+------+
|age|income|
+---+------+
|4.0|   4.0|
|5.0|   5.0|
+---+------+



## 使用统计值填充缺失值

In [11]:
# 使用Imputer将缺失值进行插值，均值
imputer = Imputer(inputCols=["age", "income"],
                  outputCols=["age", "income"]).setStrategy("mean")

df2 = imputer.fit(df).transform(df)
df2.show()

+---+------+
|age|income|
+---+------+
|1.0|   4.0|
|2.0|   4.0|
|2.0|   4.0|
|2.8|   3.0|
|4.0|   4.0|
|5.0|   5.0|
+---+------+



In [12]:
# 使用Imputer将缺失值进行插值，中位数
imputer = Imputer(inputCols=["age", "income"],
                  outputCols=["age", "income"]).setStrategy("median")

df4 = imputer.fit(df).transform(df)
df4.show()

+---+------+
|age|income|
+---+------+
|1.0|   4.0|
|2.0|   4.0|
|2.0|   4.0|
|2.0|   3.0|
|4.0|   4.0|
|5.0|   5.0|
+---+------+



In [13]:
# 使用Imputer将缺失值进行插值，众数
imputer = Imputer(inputCols=["age", "income"],
                  outputCols=["age", "income"]).setStrategy("mode")

df5 = imputer.fit(df).transform(df)
df5.show()

+---+------+
|age|income|
+---+------+
|1.0|   3.0|
|2.0|   3.0|
|2.0|   3.0|
|2.0|   3.0|
|4.0|   4.0|
|5.0|   5.0|
+---+------+



## 使用任意值填充缺失值

In [14]:
# 填充空白值为0
df3 = df.fillna(0)
df3.show()

+---+------+
|age|income|
+---+------+
|1.0|   0.0|
|2.0|   0.0|
|2.0|   0.0|
|0.0|   3.0|
|4.0|   4.0|
|5.0|   5.0|
+---+------+

